In [ ]:
from google import genai

from ai_storytime.utils import get_gemini_api_key

try:
    GEMINI_API_KEY = get_gemini_api_key()
    print("GEMINI_API_KEY loaded successfully.")
except ValueError as e:
    raise RuntimeError("GEMINI_API_KEY not found.") from e

client = genai.Client(api_key=GEMINI_API_KEY)

True

In [7]:
import textwrap
user_reading = ""
correct_reading = ""
user_prompt = f"""\
    # 指令：比較語音辨識結果與原文

    請比較以下兩段文字：

    第一段是使用者朗讀的語音辨識（ASR）結果：
    「{user_reading}」

    第二段是正確的原文：
    「{correct_reading}」

    請仔細比對，並提供回饋：
    * 如果兩者完全一致，請回覆：「讀得很好，完全正確！」。
    * 如果語音辨識結果中有任何錯誤（例如：讀錯字、漏字、多字），請明確指出錯誤的地方以及正確的內容。
        例如：「在「{{錯誤的詞}}」這個地方，你讀成了「{{辨識出的詞}}」，正確的應該是「{{正確的詞}}」。」或「你漏讀了「{{漏掉的詞}}」。」或「你多讀了「{{多出的詞}}」。」。

    請以清晰、友善的語氣提供回饋。
"""
user_prompt = textwrap.dedent(user_prompt)
print(user_prompt)

# 指令：比較語音辨識結果與原文

請比較以下兩段文字：

第一段是使用者朗讀的語音辨識（ASR）結果：
「」

第二段是正確的原文：
「」

請仔細比對，並提供回饋：
* 如果兩者完全一致，請回覆：「讀得很好，完全正確！」。
* 如果語音辨識結果中有任何錯誤（例如：讀錯字、漏字、多字），請明確指出錯誤的地方以及正確的內容。
    例如：「在「{錯誤的詞}」這個地方，你讀成了「{辨識出的詞}」，正確的應該是「{正確的詞}」。」或「你漏讀了「{漏掉的詞}」。」或「你多讀了「{多出的詞}」。」。

請以清晰、友善的語氣提供回饋。

